## req 

In [1]:
pip install -U chromadb langchain llama-index langchain_experimental langchain_openai


   ---------------------------------------- 0.0/19.6 MB ? eta -:--:--
   ----- ---------------------------------- 2.6/19.6 MB 12.6 MB/s eta 0:00:02
   ---------- ----------------------------- 5.2/19.6 MB 12.6 MB/s eta 0:00:02
   --------------- ------------------------ 7.6/19.6 MB 12.4 MB/s eta 0:00:01
   -------------------- ------------------- 10.2/19.6 MB 12.4 MB/s eta 0:00:01
   -------------------------- ------------- 12.8/19.6 MB 12.4 MB/s eta 0:00:01
   ------------------------------ --------- 15.2/19.6 MB 12.4 MB/s eta 0:00:01
   ------------------------------------ --- 17.8/19.6 MB 12.4 MB/s eta 0:00:01
   ---------------------------------------- 19.6/19.6 MB 12.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ------------- -------------------------- 2.6/7.6 MB 13.0 MB/s eta 0:00:01
   --------------------------- ------------ 5.2/7.6 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 7.6/7.6 MB 12.2 MB/s eta 0:00:00



In [2]:
import os
print(os.getcwd())  # This shows your current working directory


C:\Users\vreddy_quantum-i\Desktop\Symptoms_checker


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install pymupdf langchain


In [4]:
!pip install langchain openai tiktoken
!pip install faiss-cpu  # or faiss-gpu if needed
!pip install sentence-transformers
!pip install chromadb  # optional if using Chroma instead of FAISS
!pip install pymupdf  # for PDF reading
!pip install pypdf    # optional PDF reading alternative
!pip install python-dotenv  # to manage API keys

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


## code

In [5]:
!pip install faiss-cpu

In [6]:
import fitz  # PyMuPDF

def extract_text(path):
    with fitz.open(path) as doc:  # ensures proper file closing
        return "\n".join([page.get_text() for page in doc])

# ✅ Make sure the filename has the correct extension
text = extract_text(r"C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf")


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Convert to a Langchain Document
doc = Document(page_content=text, metadata={"source": "1100_pages.pdf"})

# Initialize splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,         # number of characters per chunk
    chunk_overlap=200        # slight overlap between chunks
)

chunks = splitter.split_documents([doc])
print(f"Total Chunks: {len(chunks)}")
print(chunks[0].page_content[:300])  # Preview first chunk
print

Total Chunks: 4175
SECTION II
Diagnostic Criteria and Codes
Neurodevelopmental Disorders
Schizophrenia Spectrum and Other Psychotic Disorders
Bipolar and Related Disorders
Depressive Disorders
Anxiety Disorders
Obsessive-Compulsive and Related Disorders
Trauma- and Stressor-Related Disorders
Dissociative Disorders
Som


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [ ]:
from sentence_transformers import SentenceTransformer

import faiss

import pickle
 
# Load model

model = SentenceTransformer("all-MiniLM-L6-v2")
 
# Assume your chunks are in a list called `chunks`

# Example: chunks = ["Depression is defined as...", "Schizophrenia criteria include..."]
 
# Embed chunks

embeddings = model.encode(chunks, show_progress_bar=True)
 
# Create FAISS index

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)

index.add(embeddings)
 
# Save index and chunks

faiss.write_index(index, "dsm5_index.faiss")

with open("dsm5_chunks.pkl", "wb") as f:

    pickle.dump(chunks, f) 

In [ ]:
import numpy as np
 
# Load index and chunks

index = faiss.read_index("dsm5_index.faiss")

with open("dsm5_chunks.pkl", "rb") as f:

    chunks = pickle.load(f)
 
# User query

query = "Patient reports low mood, fatigue, and feelings of worthlessness."
 
# Embed the query

query_embedding = model.encode([query])
 
# Search top 5 similar

D, I = index.search(np.array(query_embedding), k=5)
 
# Retrieve matched chunks

retrieved_chunks = [chunks[i] for i in I[0]]

In [9]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy as np
from transformers import pipeline
import torch
import os

# Step 1: Extract text from PDF
def extract_text(path):
    try:
        with fitz.open(path) as doc:
            return "\n".join([page.get_text() for page in doc])
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None

# Step 2: Prepare documents and chunk them
def prepare_documents(pdf_path):
    text = extract_text(pdf_path)
    if not text:
        raise ValueError("Failed to extract text from PDF")
    
    doc = Document(page_content=text, metadata={"source": pdf_path})
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents([doc])
    print(f"Total Chunks: {len(chunks)}")
    print(f"First chunk preview: {chunks[0].page_content[:300]}")
    return chunks

# Step 3: Create and save embeddings
def create_embeddings(chunks, model_name="all-MiniLM-L6-v2", index_file="dsm5_index.faiss", chunks_file="dsm5_chunks.pkl"):
    try:
        model = SentenceTransformer(model_name)
        chunk_texts = [chunk.page_content for chunk in chunks]
        embeddings = model.encode(chunk_texts, show_progress_bar=True)
        
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        
        faiss.write_index(index, index_file)
        with open(chunks_file, "wb") as f:
            pickle.dump(chunks, f)
        print(f"Saved index to {index_file} and chunks to {chunks_file}")
        return index, chunks
    except Exception as e:
        print(f"Error creating embeddings: {e}")
        return None, None

# Step 4: Load index and chunks
def load_index_and_chunks(index_file="dsm5_index.faiss", chunks_file="dsm5_chunks.pkl"):
    if os.path.exists(index_file) and os.path.exists(chunks_file):
        try:
            index = faiss.read_index(index_file)
            with open(chunks_file, "rb") as f:
                chunks = pickle.load(f)
            print(f"Loaded index from {index_file} and chunks from {chunks_file}")
            return index, chunks
        except Exception as e:
            print(f"Error loading index or chunks: {e}")
            return None, None
    return None, None

# Step 5: Retrieve relevant chunks
def retrieve_chunks(query, index, chunks, model, top_k=3):
    try:
        query_embedding = model.encode([query])[0]
        distances, indices = index.search(np.array([query_embedding]), top_k)
        return [chunks[i].page_content for i in indices[0]]
    except Exception as e:
        print(f"Error retrieving chunks: {e}")
        return []

# Step 6: Generate response
def generate_response(query, relevant_chunks, model_name="distilgpt2"):
    try:
        generator = pipeline("text-generation", model=model_name, device=0 if torch.cuda.is_available() else -1)
        context = "\n".join(relevant_chunks)
        prompt = f"Question: {query}\nContext: {context}\nAnswer in a concise and accurate manner:"
        response = generator(prompt, max_length=200, num_return_sequences=1, truncation=True)[0]["generated_text"]
        # Extract only the answer part
        answer = response.split("Answer:")[-1].strip() if "Answer:" in response else response.strip()
        return answer
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Sorry, I couldn't generate a response."

# Main function
def main(pdf_path, query):
    # Prepare documents
    chunks = prepare_documents(pdf_path)
    if not chunks:
        return "Failed to prepare documents."
    
    # Load or create embeddings
    index_file, chunks_file = "dsm5_index.faiss", "dsm5_chunks.pkl"
    index, chunks = load_index_and_chunks(index_file, chunks_file)
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    if index is None or chunks is None:
        print("Creating new embeddings...")
        index, chunks = create_embeddings(chunks, model_name="all-MiniLM-L6-v2", index_file=index_file, chunks_file=chunks_file)
        if index is None or chunks is None:
            return "Failed to create embeddings."
    
    # Retrieve and generate
    relevant_chunks = retrieve_chunks(query, index, chunks, model, top_k=3)
    if not relevant_chunks:
        return "No relevant information found."
    
    response = generate_response(query, relevant_chunks)
    return response

# Command-line interface for testing
if __name__ == "__main__":
    pdf_path = r"C:\Users\vreddy_quantum-i\Downloads\DSM-5_filtered.pdf"
    
    # Verify PDF exists
    if not os.path.exists(pdf_path):
        print(f"PDF file not found at {pdf_path}")
    else:
        while True:
            query = input("Enter your question (or 'quit' to exit): ")
            if query.lower() == "quit":
                break
            response = main(pdf_path, query)
            print(f"\nQuery: {query}")
            print(f"Response: {response}\n")

Enter your question (or 'quit' to exit):  look 


Total Chunks: 4175
First chunk preview: SECTION II
Diagnostic Criteria and Codes
Neurodevelopmental Disorders
Schizophrenia Spectrum and Other Psychotic Disorders
Bipolar and Related Disorders
Depressive Disorders
Anxiety Disorders
Obsessive-Compulsive and Related Disorders
Trauma- and Stressor-Related Disorders
Dissociative Disorders
Som
Creating new embeddings...
Error creating embeddings: 'NoneType' object is not iterable

Query: look 
Response: Failed to create embeddings.



Enter your question (or 'quit' to exit):  fever


Total Chunks: 4175
First chunk preview: SECTION II
Diagnostic Criteria and Codes
Neurodevelopmental Disorders
Schizophrenia Spectrum and Other Psychotic Disorders
Bipolar and Related Disorders
Depressive Disorders
Anxiety Disorders
Obsessive-Compulsive and Related Disorders
Trauma- and Stressor-Related Disorders
Dissociative Disorders
Som
Creating new embeddings...
Error creating embeddings: 'NoneType' object is not iterable

Query: fever
Response: Failed to create embeddings.



Enter your question (or 'quit' to exit):  fever


...

Total Chunks: 4175
First chunk preview: SECTION II
Diagnostic Criteria and Codes
Neurodevelopmental Disorders
Schizophrenia Spectrum and Other Psychotic Disorders
Bipolar and Related Disorders
Depressive Disorders
Anxiety Disorders
Obsessive-Compulsive and Related Disorders
Trauma- and Stressor-Related Disorders
Dissociative Disorders
Som
Creating new embeddings...
Error creating embeddings: 'NoneType' object is not iterable

Query: anxiety
Response: Failed to create embeddings.



Enter your question (or 'quit' to exit):  anxiety


Total Chunks: 4175
First chunk preview: SECTION II
Diagnostic Criteria and Codes
Neurodevelopmental Disorders
Schizophrenia Spectrum and Other Psychotic Disorders
Bipolar and Related Disorders
Depressive Disorders
Anxiety Disorders
Obsessive-Compulsive and Related Disorders
Trauma- and Stressor-Related Disorders
Dissociative Disorders
Som
Creating new embeddings...
Error creating embeddings: 'NoneType' object is not iterable

Query: anxiety
Response: Failed to create embeddings.



KeyboardInterrupt: Interrupted by user

Enter your question (or 'quit' to exit):  QUIT
